# Import Libraries

Steps
1. Import libraries
2. Create S3
3. Mapping train and test data in s3
4. mapping the path of the models in S3

In [21]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [22]:
bucket_name = 'bankapplication123456' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-west-1


In [23]:
#create a bucket (you can also do this manually)
s3 = boto3.resource('s3')
try:
    if my_region =='global':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket creasted successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket creasted successfully


In [24]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication123456/xgboost-as-a-built-in-algo/output


download dataset and store into S3 bucket

In [31]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://raw.githubusercontent.com/Joseph-Morris/Build-train-deploy-ML-model-in-AWS-Sagemaker/main/bank_clean.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [26]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [28]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [31]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

Building Models Xgboot- Inbuilt Algorithm


In [36]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name,version ="latest")

In [37]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [58]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_count=1,
                                          instance_count=1,
                                          instance_type='ml.m5.2xlarge', 
                                          train_type='ml.m5.2xlarge', 
                                          train_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot=True,
                                          train_run=300,
                                          train_wait=600)

In [46]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2022-04-21 20:15:39 Starting - Starting the training job...
2022-04-21 20:16:02 Starting - Preparing the instances for trainingProfilerReport-1650572138: InProgress
......
2022-04-21 20:17:08 Downloading - Downloading input data
2022-04-21 20:17:08 Training - Downloading the training image......
2022-04-21 20:18:03 Training - Training image download completed. Training in progress..Arguments: train
[2022-04-21:20:18:01:INFO] Running standalone xgboost training.
[2022-04-21:20:18:01:INFO] File size need to be processed in the node: 4.83mb. Available memory size in the node: 23533.29mb
[2022-04-21:20:18:01:INFO] Determined delimiter of CSV input is ','
[20:18:01] S3DistributionType set as FullyReplicated
[20:18:01] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-04-21:20:18:01:INFO] Determined delimiter of CSV input is ','
[20:18:01] S3DistributionType set as FullyReplicated
[20:18:01] 12357x59 matrix with 729063 entri

Deploy Machine Learning Model As Endpoints

In [47]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

------!

Prediction of the Test Data

In [60]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


In [61]:
#output of all the test data
predictions_array

array([0.05578489, 0.04585116, 0.05088526, ..., 0.03450042, 0.03321051,
       0.03635123])

In [62]:
#confusion metrics
#tn = truenegative, fp = false positive etc
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10784)    36% (152)
Purchase        10% (1149)     64% (272) 



deleting the endpoints THIS WILL DELETE YOUR ENDPOINT!

In [64]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-west-1:656376156636:endpoint/xgboost-2022-04-21-21-43-37-016".